In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch.helpers import parallel_bulk
import json
es = Elasticsearch(timeout=240)

In [2]:
#Tener en cuenta de que, si se ejecuta varias veces el codigo, se subiran varios tweets varias veces
import os
from os.path import join

errors = 0
path = 'D:\\Tweets\\2018'

%load_ext jupyternotify
for path, folders, files in os.walk(path, onerror = None, topdown=False):
    for file in files:
        if not (file == 'log'):
            print(path + '\\' + file)
            with open(path + '\\' + file, "r") as fobj:
                list_bodies = fobj.readlines()
                helpers.bulk(es, list_bodies, index='tweets')
print(errors)
%notify

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
#Tener en cuenta de que, si se ejecuta varias veces el codigo, se subiran varios tweets varias veces
import os
from os.path import join

errors = 0
path = 'D:\\Tweets\\2018\\g20_18-11-11-02-38\\'

%load_ext jupyternotify
for path, folders, files in os.walk(path, onerror = None, topdown=False):
    for file in files:
        if not (file == 'log'):
            print(path + '\\' + file)
            with open(path + '\\' + file, "r") as fobj:
                list_bodies = fobj.readlines()
                helpers.bulk(es, list_bodies, index='tweets')
print(errors)
%notify

In [ ]:
#Tener en cuenta de que, si se ejecuta varias veces el codigo, se subiran varios tweets varias veces
import os
from os.path import join

errors = 0
path = 'D:\\Tweets\\2018'

%load_ext jupyternotify
for path, folders, files in os.walk(path):
    try:
        for file in files:
            if not (file == 'log'):
                with open(join(path, file),"r") as fobj:
                    list_bodies = fobj.readlines()
                    helpers.bulk(es, list_bodies, index='tweets')
    except:
        errors+=1
        #Cada vez que no se encuentra un archivo FileNotFoundError
        #O hay un error al indexar los tweets (BulkIndexError?) se ejecuta este codigo
        #print('File Not Found UwU\n')
print(errors)
%notify

In [ ]:
#Tener en cuenta de que, si se ejecuta varias veces el codigo, se subiran varios tweets varias veces
import os
from os.path import join

def gendata(lines):
    for line in lines:
        if not (line.startswith('{"index"')):
            yield {
                "_op_type": "index",
                "_index": "tweets",
                "_source": line
            }

errors = 0
#path = 'D:\\Tweets\\2021\\2021-03-30\\2021.03.30-\\g20_21-04-05-07-26'
path = 'D:\\Tweets\\2018'
#path = 'D:\\Tweets\\2020\\Marzo-Abril\\g20_20-03-03-19-05'

%load_ext jupyternotify
file_errors = open('indexErrors.txt','a', encoding='utf-8')

for path, folders, files in os.walk(path, onerror = None, topdown = False):
    try:
        for file in files:
            if not (file == 'log'):
                print(path + '\\' + file)
                with open(path + '\\' + file, "r", encoding= 'cp850') as fobj:
                    #helpers.bulk(es, list_bodies, index='prueba_tweets')
                    #actions = [json.loads(row) for row in fobj.readlines()]
                    for success, info in parallel_bulk(es, gendata(fobj.readlines())):
                    #for success, info in parallel_bulk(es, actions):
                        if not success:
                            print('A document failed:', info)
                            errors = errors + 1
    except Exception as e:
        errors = errors + 1
        file_errors.write(str(e) + '\n*******************\n')

file_errors.write('\n\n Cantidad total de errores: ' + str(errors) + '\n\n')        
file_errors.close()
        

%notify

In [ ]:
#Tener en cuenta de que, si se ejecuta varias veces el codigo, se subiran varios tweets varias veces
import os
from os.path import join

path = 'D:\\Tweets\\2020'
folderReference = 'D:\\Tweets\\2020\\Marzo-Abril\\g20_20-04-11-05-56'
l = list()

for path, folders, files in os.walk(path, onerror = None, topdown = False):
    for file in files:
        if not (file == 'log') and (path > folderReference):
            l.append(path + '\\' + file)
            #print(path + '\\' + file)       

In [ ]:
len(l)

In [ ]:
l[]

In [ ]:
#Tener en cuenta de que, si se ejecuta varias veces el codigo, se subiran varios tweets varias veces
import os
from os.path import join

def gendata(lines):
    for line in lines:
        if not (line.startswith('{"index"')):
            yield {
                "_op_type": "index",
                "_index": "tweets",
                "_source": line
            }
%load_ext jupyternotify
file_errors = open('indexErrors.txt','a', encoding='utf-8')
errors = 0

for filepath in l:
    try:
        print(filepath)
        with open(filepath, "r", encoding= 'cp850') as fobj:
            for success, info in parallel_bulk(es, gendata(fobj.readlines())):
                if not success:
                    print('A document failed:', info)
                    errors = errors + 1
    except Exception as e:
        errors = errors + 1
        file_errors.write(str(e) + '\n*******************\n')

file_errors.write('\n\n Cantidad total de errores: ' + str(errors) + '\n\n')        
file_errors.close()
%notify